# test a model

Test youy rmodel right after training in a kubeflow pipeline, or run this function independently.  In addition, the plotting components in **[test_classifier.py]()** can also be run independently.

In [16]:
import mlrun
import os
import numpy as np
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

## parameters

In [21]:
CODE_BASE          = '/User/repos/functions'

MODEL_FILE         = '/User/mlrun/models/lgb-classifier.pkl'

TARGET_DATA_PATH   = '/User/mlrun/models'
XTEST_FILE         = '/User/mlrun/models/xtest.pqt'
YTEST_FILE         = '/User/mlrun/models/ytest.pqt'

## run tests

In [22]:
yaml_name = os.path.join(CODE_BASE, 'evaluation', 'test-classifier.yaml')
if not os.path.isfile(yaml_name):
    testfn = mlrun.code_to_function(
        kind='job', 
        image='yjbds/mlrun-ds:latest',
        filename=os.path.join(CODE_BASE, 'evaluation', 'test-classifier.py'))
    testfn.build_config(base_image='yjbds/mlrun-ds:latest', commands=[])

    testfn.export(os.path.join(CODE_BASE, 'evaluation', 'test-classifier.yaml'))

In [23]:
testfn = mlrun.import_function(
    os.path.join(CODE_BASE, 'evaluation', 'test-classifier.yaml')
).apply(mlrun.mount_v3io())

In [24]:
testfn.deploy(skip_deployed=True, with_mlrun=False)

'ready'

In [25]:
task = mlrun.NewTask()
task.with_params(
    model=MODEL_FILE,
    xtest=XTEST_FILE,
    ytest=YTEST_FILE,
    target_path=TARGET_DATA_PATH)

In [26]:
tsk_run = testfn.run(task, handler='test_model')

[mlrun] 2020-01-26 14:37:14,256 starting run test_model uid=6f193cffb1714302ae174b06ca9538f6  -> http://mlrun-api:8080
[mlrun] 2020-01-26 14:37:14,354 Job is running in the background, pod: test-model-x4z5p
[mlrun] 2020-01-26 14:37:24,867 log artifact roc.html at roc.html, size: 32259, db: Y
[mlrun] 2020-01-26 14:37:25,080 log artifact confusion_matrix.html at confusion_matrix.html, size: 16188, db: Y
[mlrun] 2020-01-26 14:37:25,667 log artifact feature-importances-plot.html at feature-importances-plot.html, size: 68260, db: Y
[mlrun] 2020-01-26 14:37:25,682 log artifact feature-importances-table at /User/mlrun/models/feature-importances-table.csv, size: None, db: Y

[mlrun] 2020-01-26 14:37:25,739 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...9538f6,0,Jan 26 14:37:20,completed,test-classifier,host=test-model-x4z5pkind=jobowner=admin,,model=/User/mlrun/models/lgb-classifier.pkltarget_path=/User/mlrun/modelsxtest=/User/mlrun/models/xtest.pqtytest=/User/mlrun/models/ytest.pqt,,roc.htmlconfusion_matrix.htmlfeature-importances-plot.htmlfeature-importances-table


to track results use .show() or .logs() or in CLI: 
!mlrun get run 6f193cffb1714302ae174b06ca9538f6  , !mlrun logs 6f193cffb1714302ae174b06ca9538f6 
[mlrun] 2020-01-26 14:37:33,579 run executed, status=completed
